# eICU-CRD tutorial

This notebook shows examples of how to use the cycquery tool on [eicu-CRD](https://eicu-crd.mit.edu/).

Each query is limit to 100 rows (for quick results).

* First, setup the eICU database according to the instructions in [eicu-code](https://github.com/MIT-LCP/eicu-code/tree/master/build-db/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `EICUQuerier` accordingly.

## Imports and instantiate `EICUQuerier`

In [1]:
"""eICU-CRD tutorial."""

import cycquery.ops as qo
from cycquery import EICUQuerier


querier = EICUQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="eicu",
    user="postgres",
    password="pwd",
)
# List all tables.
querier.list_tables("eicu_crd")

2024-04-17 15:12:30,977 INFO cycquery.orm    - Database setup, ready to run queries!


['eicu_crd.admissiondrug',
 'eicu_crd.admissiondx',
 'eicu_crd.allergy',
 'eicu_crd.apacheapsvar',
 'eicu_crd.apachepatientresult',
 'eicu_crd.apachepredvar',
 'eicu_crd.careplancareprovider',
 'eicu_crd.careplaneol',
 'eicu_crd.careplangeneral',
 'eicu_crd.careplangoal',
 'eicu_crd.careplaninfectiousdisease',
 'eicu_crd.customlab',
 'eicu_crd.diagnosis',
 'eicu_crd.hospital',
 'eicu_crd.infusiondrug',
 'eicu_crd.intakeoutput',
 'eicu_crd.lab',
 'eicu_crd.medication',
 'eicu_crd.microlab',
 'eicu_crd.note',
 'eicu_crd.nurseassessment',
 'eicu_crd.nursecare',
 'eicu_crd.nursecharting',
 'eicu_crd.pasthistory',
 'eicu_crd.patient',
 'eicu_crd.physicalexam',
 'eicu_crd.respiratorycare',
 'eicu_crd.respiratorycharting',
 'eicu_crd.treatment',
 'eicu_crd.vitalaperiodic',
 'eicu_crd.vitalperiodic']

## Example 1. Get all female patients discharged in 2014 (limit to 100 rows).

In [2]:
ops = qo.Sequential(
    qo.ConditionEquals("hospitaldischargeyear", 2014),
    qo.ConditionEquals("gender", "Female"),
)
patients = querier.eicu_crd.patient()
patients = patients.ops(ops).run(limit=100)
print(f"{len(patients)} rows extracted!")

2024-04-17 15:12:31,857 INFO cycquery.orm    - Query returned successfully!


2024-04-17 15:12:31,859 INFO cycquery.utils.profile - Finished executing function run_query in 0.054049 s


100 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in `diagnosisstring`), discharged in the year 2015.

In [3]:
patients = querier.eicu_crd.patient()
diagnoses = querier.eicu_crd.diagnosis()
diagnoses = diagnoses.ops(qo.ConditionSubstring("diagnosisstring", "schizophrenia"))
patient_diagnoses = patients.join(
    join_table=diagnoses,
    on="patientunitstayid",
)
patient_diagnoses = patient_diagnoses.run(limit=100)
print(f"{len(patient_diagnoses)} rows extracted!")

2024-04-17 15:12:31,963 INFO cycquery.orm    - Query returned successfully!


2024-04-17 15:12:31,965 INFO cycquery.utils.profile - Finished executing function run_query in 0.084577 s


100 rows extracted!


## Example 3. Get potassium lab tests for patients discharged in the year 2014, for all teaching hospitals.

In [4]:
hospitals = querier.eicu_crd.hospital()
hospitals = hospitals.ops(qo.ConditionEquals("teachingstatus", True))
patients = querier.eicu_crd.patient()
patients = patients.ops(qo.ConditionEquals("hospitaldischargeyear", 2015))
patients = patients.join(
    join_table=hospitals,
    on="hospitalid",
)
labs = querier.eicu_crd.lab()
labs = labs.ops(qo.ConditionEquals("labname", "potassium"))
patient_labs = patients.join(
    join_table=labs,
    on="patientunitstayid",
).run(limit=100)
print(f"{len(patient_labs)} rows extracted!")

2024-04-17 15:12:32,037 INFO cycquery.orm    - Query returned successfully!


2024-04-17 15:12:32,038 INFO cycquery.utils.profile - Finished executing function run_query in 0.043745 s


100 rows extracted!


## Example 4. Get glucose medications (substring search) for female patients discharged in 2014.

In [5]:
ops = qo.Sequential(
    qo.ConditionEquals("hospitaldischargeyear", 2014),
    qo.ConditionEquals("gender", "Female"),
)
patients = querier.eicu_crd.patient()
patients = patients.ops(ops)
medications = querier.eicu_crd.medication()
medications = medications.ops(qo.ConditionSubstring("drugname", "glucose"))
patient_medications = patients.join(
    join_table=medications,
    on="patientunitstayid",
).run(limit=100)
print(f"{len(patient_medications)} rows extracted!")

2024-04-17 15:12:32,221 INFO cycquery.orm    - Query returned successfully!


2024-04-17 15:12:32,222 INFO cycquery.utils.profile - Finished executing function run_query in 0.162660 s


100 rows extracted!
